In [1]:
import sys
import os

path = os.path.abspath('..')
sys.path.append(path + '/src')
path = '/'.join(path.split('/')[:-1])
sys.path.append(path + '/get_issues/src')

In [2]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt  
plt.style.use('dark_background')
%matplotlib inline

## Загрузка данных о репозиториях

Данные содержат информацию о репозиториях, имеющих преобладающий язык Python, с активными страницами инцидентов.

In [3]:
path_to_repos_list = '../data/'
repos_list_filename = 'python_repos_interesting.json'

In [4]:
from collect_repos import load_repo_info_from_json
repositories = load_repo_info_from_json(path_to_repos_list,
                                        repos_list_filename)

In [5]:
repositories.columns

Index(['databaseId', 'nameWithOwner', 'description', 'mirrorUrl', 'createdAt',
       'pushedAt', 'updatedAt', 'forkCount', 'stargazerCount',
       'hasIssuesEnabled', 'isArchived', 'isDisabled', 'isFork', 'isMirror',
       'diskUsage', 'parent', 'defaultBranchRef.name',
       'commitComments.totalCount', 'licenseInfo.name', 'watchers.totalCount',
       'issues.totalCount', 'pullRequests.totalCount', 'primaryLanguage.name',
       'releases.totalCount', 'languages.edges', 'licenseInfo'],
      dtype='object')

In [6]:
repositories = repositories.astype({'forkCount':'int32', 'stargazerCount':'int32', 
                                    'diskUsage':'float64', 'commitComments.totalCount':'int32',
                                    'watchers.totalCount':'int32', 'issues.totalCount':'int32', 
                                    'pullRequests.totalCount':'int32', 'releases.totalCount':'int32'})

## Конечная цель фильтрации репозиториев

Конечной целью фильтрации репозиториев является выбор репозиториев согласно следующим критериям:

- популярность и количество разработчиков (контрибуторов, майнтейнеров и коммитеров)
- репозиторий должен быть хранилищем версионированного исходного кода, предназначенного для разработки ПО, не для хранения, веб-хостинга, учебных целей и т.п.
- должен содержать как можно больше "хороших" сообщений об инцидентах с меткой bug или похожей меткой
- должен быть активно разрабатываемым
- должен иметь релевантную лицензию

## Первичный анализ данных о репозиториях

Посмотрим на описательные статистики основных числовых параметров репозиториев:

In [6]:
repositories.describe()

,databaseId,forkCount,stargazerCount,diskUsage,commitComments.totalCount,watchers.totalCount,issues.totalCount,pullRequests.totalCount,releases.totalCount,licenseInfo
count,1.490250e+05,149025.000000,149025.000000,1.490250e+05,149025.000000,149025.000000,149025.000000,149025.000000,149025.000000,0.0
mean,1.032620e+08,35.686187,133.404717,1.812600e+04,2.359450,12.534488,33.485093,37.757806,2.457011,NaN
std,8.320557e+07,284.363675,1066.370635,2.183703e+05,38.241831,58.664404,288.999818,301.628020,13.072654,NaN
min,8.338000e+03,1.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000,1.000000,0.000000,NaN
25%,3.009696e+07,2.000000,4.000000,8.100000e+01,0.000000,2.000000,2.000000,2.000000,0.000000,NaN
50%,8.674015e+07,6.000000,13.000000,3.790000e+02,0.000000,4.000000,6.000000,5.000000,0.000000,NaN
75%,1.606999e+08,18.000000,50.000000,3.630000e+03,0.000000,10.000000,18.000000,17.000000,1.000000,NaN
max,3.242041e+08,43282.000000,118425.000000,4.557675e+07,9435.000000,6043.000000,69367.000000,44303.000000,1661.000000,NaN


## Первичная фильтрация репозиториев

Отберем репозитории по совокупности из четырех параметров: число форков, число звезд, число инцидентов и число пулл-реквестов. Данные параметры отражают относительную популярность репозитория, а также интенсивность его развития (с момента создания).
Это должно позволить сузить круг рассматриваемых репозиториев, по которым будет проводиться дальнейшая фильтрация. Зададим в качестве порогов соответствующих верхних ограничений 75%-процентили.

In [7]:
quantile_threshold = 0.75
selected_repositories = repositories.loc[(repositories['stargazerCount']>=repositories['stargazerCount'].quantile(q=quantile_threshold)) &
                                         (repositories['issues.totalCount']>=repositories['issues.totalCount'].quantile(q=quantile_threshold)) &
                                         (repositories['pullRequests.totalCount']>=repositories['pullRequests.totalCount'].quantile(q=quantile_threshold))]
selected_repositories.shape

(12201, 26)

In [9]:
repositories['stargazerCount'].quantile(q=quantile_threshold), repositories['issues.totalCount'].quantile(q=quantile_threshold), repositories['pullRequests.totalCount'].quantile(q=quantile_threshold)

(50.0, 18.0, 17.0)

In [8]:
selected_repositories.describe()

,databaseId,forkCount,stargazerCount,diskUsage,commitComments.totalCount,watchers.totalCount,issues.totalCount,pullRequests.totalCount,releases.totalCount,licenseInfo
count,1.220100e+04,12201.000000,12201.000000,1.220100e+04,12201.000000,12201.000000,12201.000000,12201.000000,12201.000000,0.0
mean,6.841537e+07,234.735022,1013.581674,3.296140e+04,16.566757,56.689943,235.975576,240.280305,10.610360,NaN
std,6.925391e+07,945.949864,3532.483385,4.305154e+05,125.518285,173.043529,962.284793,983.422413,30.919353,NaN
min,8.338000e+03,1.000000,50.000000,9.000000e+00,0.000000,0.000000,18.000000,17.000000,0.000000,NaN
25%,1.209314e+07,34.000000,109.000000,4.670000e+02,0.000000,13.000000,41.000000,32.000000,0.000000,NaN
50%,4.426067e+07,69.000000,248.000000,2.139000e+03,2.000000,24.000000,80.000000,65.000000,2.000000,NaN
75%,1.061548e+08,165.000000,715.000000,1.306200e+04,8.000000,50.000000,185.000000,171.000000,11.000000,NaN
max,3.136742e+08,43282.000000,118425.000000,4.411732e+07,9435.000000,6043.000000,69367.000000,44303.000000,1305.000000,NaN


In [10]:
selected_repositories['nameWithOwner'].to_csv('../data/interesting_repos.csv', 
                                              sep='\n', 
                                              index=False, 
                                              header=False)